# [4] Ollama

- 본 섹션은 Ollama를 활용하여 LLM 모델을 통해 예측을 수행합니다.

## 4.1. Ollama 시작하기

- Ollama는 다양한 오픈소스 LLM 모델을 로컬에서 쉽게 사용할 수 있도록 도와주는 프로그램입니다.

### 4.1.1. Ollama 설치

<https://ollama.com/download>

위 링크에서 Ollama를 다운로드 받아 설치합니다.

### 4.1.2. 모델 다운로드

- 다음 파이썬 코드를 통해 `llama3.2-vision:11b` 모델을 다운로드합니다.

In [2]:
import ollama

model_list = [
    "llama3.2-vision:11b", 
    "llava:13b", 
    "minicpm-v:8b", 
    "llava-llama3:8b"
]

for model in model_list:
    print("Pulling", model)
    ollama.pull(model)

# print("Pulling llma3.2-vision:11b")
# ollama.pull("llama3.2-vision:11b")
# print("Pulling llava:13b")
# ollama.pull("llava:13b")
# print("Pulling minicpm-v:8b")
# ollama.pull("minicpm-v:8b")
# print("Pulling llava-llama3:8b")
# ollama.pull("llava-llama3:8b")

Pulling llama3.2-vision:11b
Pulling llava:13b
Pulling minicpm-v:8b
Pulling llava-llama3:8b


## 4.2. Llama 모델로 예측하기

### 4.2.1. Vehicle API 클라이언트 로드

In [ ]:
import os
from dotenv import load_dotenv
from aicastle.deepracer.vehicle.api.client import VehicleClient
load_dotenv(".env")

vehicle =VehicleClient(
    ip=os.getenv("VEHICLE_IP"),
    password=os.getenv("VEHICLE_PASSWORD"),
)


### 4.2.2. 변수 및 프롬프트 정의

In [5]:
from aicastle.deepracer.vehicle.api.utils import read_image, get_image_base64

image_width = 160
image_height = 120

messages_prompt = [
    {
        "role": "system",
        "content": (
            "You are an autonomous driving AI."
            "Analyze the image captured by the front camera and determine the direction."
            "Answer only with one of: straight, left, or right."
        )
    },
    {
        "role": "user",
        "content": "차선을 이탈하거나 상자(장애물)과 충돌하지 않게 주행하세요."
    },
    {
        "role": "user",
        "images":[
            get_image_base64(
                read_image(
                    'sample/images/1_right.png', 
                    width=image_width, 
                    height=image_height,
                    color="rgb"
                ),
                format="jpg"
            )
        ]
    },
    {
        "role": "assistant",
        "content": "right"
    },
    {
        "role": "user",
        "images":[
            get_image_base64(
                read_image(
                    'sample/images/2_straight.png', 
                    width=image_width, 
                    height=image_height,
                    color="rgb"
                ),
                format="jpg"
            )
        ]
    },
    {
        "role": "assistant",
        "content": "straight"
    },
    {
        "role": "user",
        "images":[
            get_image_base64(
                read_image(
                    'sample/images/3_left.png', 
                    width=image_width, 
                    height=image_height,
                    color="rgb"
                ),
                format="jpg"
            )
        ]
    },
    {
        "role": "assistant",
        "content": "left"
    },
]

### 3.2.3. 예측 및 자율주행

In [ ]:
from IPython.display import clear_output
import time
import ollama
from ollama import Client, chat
# client = Client(host='http://localhost:11434',headers={'x-some-header': 'some-value'})

model="llama3.2-vision:11b"
repeat_count = 10
user_message = """

""".strip()

target_actions = {
    "straight":{"angle": 0, "speed": 2},
    "left":{"angle": 30, "speed": 2},
    "right":{"angle": -30, "speed": 2},
}


vehicle.set_speed_percent(60)
try :
    for repeat_idx in range(repeat_count):
        # image = read_image("sample/images/test.png", width=image_width, height=image_height, color="rgb")
        image = vehicle.get_image(width=image_width, height=image_height, color="rbg")
        image_base64 = get_image_base64(image, format="jpg")
        messages = messages_prompt + [
            {
                "role": "user",
                "content": user_message,
                "images":[image_base64]
            }
        ]

        s_time = time.time()
        # response = client.chat(
        response = chat(
            model=model,
            messages=messages,
            options={
                "num_predict": 1,
                "top_p": 0, 
                "temperature": 0,
                # "num_gpu": 1,
            },
        )
        f_time = time.time()
        
        content = response["message"]["content"]
        if content in target_actions:
            angle = target_actions[content]["angle"]
            speed = target_actions[content]["speed"]
        else :
            angle = 0
            speed = 0
        
        print(f"[repeat_idx] {repeat_idx+1}/{repeat_count}")
        print(f"[inference_time] {f_time - s_time:.4f} sec")
        print(f"[content] {content}")
        print(f"[angle] {angle}")
        print(f"[speed] {speed}")
        clear_output(wait=True)

        vehicle.move(angle=angle, speed=speed)

    vehicle.stop()
except :
    vehicle.stop()